In [ ]:
# References:
# Kai Tao, Tianze Liu, Jieyuan Ning, Fenglin Niu, "Estimating sedimentary and crustal structure
# using wavefield continuation: theory, techniques and applications", Geophysical Journal International,
# Volume 197, Issue 1, April, 2014, Pages 443-457, https://doi.org/10.1093/gji/ggt515

--------------------------------------------------------

--------------------------------

# Import required libraries and run case studies

In [ ]:
import logging

import numpy as np
import matplotlib.pyplot as plt
# import scipy.signal as signal
from scipy import stats

import h5py
import obspy
import obspyh5

from seismic.stream_quality_filter import curate_stream3c
from seismic.receiver_fn.rf_util import compute_vertical_snr
from seismic.receiver_fn.rf_util import KM_PER_DEG
from seismic.receiver_fn.rf_synthetic import synthesize_ideal_seismogram

In [ ]:
from seismic.inversion.wavefield_decomp.wavefield_continuation_tao import WfContinuationSuFluxComputer
from seismic.inversion.wavefield_decomp.model_properties import LayerProps
from seismic.inversion.wavefield_decomp.wfd_plot import plot_Esu_space
from seismic.inversion.wavefield_decomp.network_event_dataset import NetworkEventDataset

In [ ]:
from tqdm.auto import tqdm
from joblib import Parallel, delayed

-----------------------------

## Run using on-demand synthetic data to validate implementation

In [ ]:
network = 'AU'
target_station = 'QIS'

In [ ]:
f_s = 10
data_synth = synthesize_ideal_seismogram(network, target_station, 'velocity', 65, 140, f_s=f_s,
                                         sourcedepthmetres=0)
# data_synth

In [ ]:
data_synth.plot(type='relative', reftime=data_synth[0].stats.onset, outfile='synth_event.png', dpi=300)

In [ ]:
# Time window to trim input traces to
TIME_WINDOW = (-20, 50)
# Time window used for integration of energy flux
FLUX_WINDOW = (-10, 20)
# Snippet around onset to use for processing
CUT_WINDOW = (-5, 30)

In [ ]:
t_onset = data_synth[0].stats.onset - data_synth[0].stats.starttime

In [ ]:
data_all = {'synth_event_0': data_synth}

In [ ]:
flux_computer = WfContinuationSuFluxComputer(data_all.values(), f_s, TIME_WINDOW, CUT_WINDOW)

In [ ]:
# Define bulk properties of mantle (lowermost half-space)
mantle_props = LayerProps(8.0, 4.5, 3.3, np.Infinity)

In [ ]:
# Define single layer earth model (crust over mantle only, no sediment)
# Vs here is postulated.
# H here is postulated.
earth_props = np.array([LayerProps(6.1, 3.7, 2.7, 35.0)])

In [ ]:
energy, energy_per_event, mantle_wave_components = flux_computer(mantle_props, earth_props, flux_window=FLUX_WINDOW)

In [ ]:
print(energy)

In [ ]:
# Plot the wavefield decomposition at the top of the mantle
plt.figure(figsize=(16,12))
t = flux_computer.times()
plt.plot(t, mantle_wave_components[0,0,:], label='$P_{down}$', alpha=0.8, linewidth=2)
plt.plot(t, mantle_wave_components[0,1,:], label='$P_{up}$', alpha=0.8, linewidth=2)
plt.plot(t, mantle_wave_components[0,2,:], label='$S_{down}$', alpha=0.8, linewidth=2)
plt.plot(t, mantle_wave_components[0,3,:], label='$S_{up}$', alpha=0.8, linewidth=2)
plt.plot(t, np.sum(mantle_wave_components[0,:,:], axis=0), color='#40404080', label='Total', linewidth=4)
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Amplitude (normalized)', fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(linestyle=':', color="#80808080")
plt.title("Waveform at top of mantle", fontsize=20, y=1.01)
plt.legend(fontsize=14)
plt.savefig('synth_waveform_top_mantle.png', dpi=300)
plt.show()

In [ ]:
# Define grid search space
H = np.linspace(20, 70, 251)
k = np.linspace(1.5, 2.1, 101)

In [ ]:
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 0, H, k, (-5, 10))
plot_Esu_space(H_grid, k_grid, Esu, title='$E_{{SU}}$ over full range, synthetic seismogram (-5, 10)',
               c_range=(0, 0.5), savefile_name='Esu_synthetic_full_param_range_(-5,10)')

In [ ]:
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 0, H, k, (-10, 15))
plot_Esu_space(H_grid, k_grid, Esu, title='$E_{{SU}}$ over full range, synthetic seismogram (-10, 15)',
               c_range=(0, 0.5), savefile_name='Esu_synthetic_full_param_range_(-10,15)')

In [ ]:
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 0, H, k, (-10, 20))
plot_Esu_space(H_grid, k_grid, Esu, title='$E_{{SU}}$ over full range, synthetic seismogram (-10, 20)',
               c_range=(0, 0.5), savefile_name='Esu_synthetic_full_param_range_(-10,20 same as Tao)')

In [ ]:
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 0, H, k, (-20, 20))
plot_Esu_space(H_grid, k_grid, Esu, title='$E_{{SU}}$ over full range, synthetic seismogram (-20, 20)',
               c_range=(0, 0.5), savefile_name='Esu_synthetic_full_param_range_(-20,20)')

In [ ]:
def find_energy_minimum_location(energy, h_grid, k_grid):
    min_loc = np.unravel_index(np.argmin(energy), energy.shape)
    H_min = h_grid[0, min_loc[1]]
    k_min = k_grid[min_loc[0], 0]
    return (H_min, k_min)
# end func

In [ ]:
find_energy_minimum_location(Esu, H_grid, k_grid)

------------------------

## Run on data converted from Tao's SAC files

Replicate work of Tao on NE68

In [ ]:
network = 'BD'
target_station = 'NE68'

In [ ]:
# Resampling rate
f_s = 10.0  # Matches dt==0.1 used by Tao
# Time window of original data to use for processing. All traces must have at least this extent
# about the onset time.
TIME_WINDOW = (-20, 50)
# Cut window for selecting central wavelet
CUT_WINDOW = (-5, 30)
# Narrower time window used for integration of energy flux
FLUX_WINDOW = (-10, 20)

In [ ]:
# src_file = (r"/g/data/ha3/am7399/dev/RFsediment/YP.NE68/H-beta_SCM_Esu_DCmatlab_station/sac"
#             r"/event_test3.use.hdf5")
src_file = (r"/g/data/ha3/am7399/dev/RFsediment/YP.NE68/H-beta_SCM_Esu_DCmatlab_station/sac"
            r"/event.use.h5")

In [ ]:
traces = obspy.read(src_file, 'H5')

In [ ]:
data_all = NetworkEventDataset(traces, network, target_station, ordering='ZRT')

In [ ]:
# Define bulk properties of mantle (lowermost half-space)
mantle_props = LayerProps(8.0, 4.5, 3.3, np.Infinity)
mantle_props

In [ ]:
earth_props = np.array([LayerProps(2.1, 0.5, 1.97, 0.3), LayerProps(6.4, 3.7, 2.7, 35.0)])
earth_props

In [ ]:
flux_computer = WfContinuationSuFluxComputer(data_all.station(target_station).values(), f_s, TIME_WINDOW, CUT_WINDOW)

In [ ]:
energy, energy_per_event, mantle_wave_components = flux_computer(mantle_props, earth_props, flux_window=FLUX_WINDOW)

In [ ]:
print(energy)

### Perform grid search on sediment properties

In [ ]:
H_sed = np.linspace(0, 1.5, 101)
k_sed = np.linspace(1.6, 7.0, 101)
TIME_WINDOW = (-10, 20)
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 0, H_sed, k_sed, TIME_WINDOW)
plot_Esu_space(H_grid, k_grid, Esu, title='Sediment properties grid search, iteration 1')

In [ ]:
H_sediment, k_sediment = find_energy_minimum_location(Esu, H_grid, k_grid)
print(H_sediment, k_sediment)

### Perform grid search on crust properties

In [ ]:
sediment_props = LayerProps(2.1, 2.1/k_sediment, 1.97, H_sediment)
sediment_props

In [ ]:
earth_props[0] = sediment_props

In [ ]:
H_crust = np.linspace(25, 45, 101)
k_crust = np.linspace(1.5, 2.1, 101)
TIME_WINDOW = (-10, 20)
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 1, H_crust, k_crust, TIME_WINDOW)
plot_Esu_space(H_grid, k_grid, Esu, title='Crust properties grid search, iteration 1')

In [ ]:
H_crust, k_crust = find_energy_minimum_location(Esu, H_grid, k_grid)
print(H_crust, k_crust)

### Repeat grid search on sediment properties (2nd iteration)

In [ ]:
crust_props = LayerProps(6.4, 6.4/k_crust, 2.7, H_crust)
earth_props[1] = crust_props

In [ ]:
H_sed = np.linspace(0, 1.5, 101)
k_sed = np.linspace(1.6, 7.0, 101)
TIME_WINDOW = (-10, 20)
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 0, H_sed, k_sed, TIME_WINDOW)
plot_Esu_space(H_grid, k_grid, Esu, title='Sediment properties grid search, iteration 2')

In [ ]:
H_sediment, k_sediment = find_energy_minimum_location(Esu, H_grid, k_grid)
print(H_sediment, k_sediment)

### Repeat grid search on crust properties (2nd iteration)

In [ ]:
sediment_props = LayerProps(2.1, 2.1/k_sediment, 1.97, H_sediment)
earth_props[0] = sediment_props

In [ ]:
H_crust = np.linspace(25, 45, 101)
k_crust = np.linspace(1.5, 2.1, 101)
TIME_WINDOW = (-10, 20)
H_grid, k_grid, Esu = flux_computer.grid_search(mantle_props, earth_props, 1, H_crust, k_crust, TIME_WINDOW)
plot_Esu_space(H_grid, k_grid, Esu, title='Crust properties grid search, iteration 2')

In [ ]:
# Extract final minimum H_crust and Vs_crust
H_crust, k_crust = find_energy_minimum_location(Esu, H_grid, k_grid)
print(H_crust, k_crust)

----------------------------------------